# Setup

From: https://platform.openai.com/docs/guides/text-generation/chat-completions-api

The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content. Conversations can be as short as one message or many back and forth turns.

How to set openai API key if using conda environments:
https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#setting-environment-variables

1. `conda activate [name of env]`
2. `conda env config vars set OPENAI_API_KEY=[openai api key]`
3. `conda activate [name of env]`
4. restart vscode/kernel

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# # deprecated
# import openai
# openai.api_key = os.environ["OPENAI_API_KEY"]

# def llm(prompt, stop=["\n"]):
#     response = openai.Completion.create(
#       model="text-davinci-002",
#       prompt=prompt,
#       temperature=0,
#       max_tokens=100,
#       top_p=1,
#       frequency_penalty=0.0,
#       presence_penalty=0.0,
#       stop=stop
#     )
#     return response["choices"][0]["text"]


In [3]:
from openai import OpenAI
client = OpenAI()

# response = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Who won the world series in 2020?"},
#     {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
#     {"role": "user", "content": "Where was it played?"}
#   ]
# )

In [26]:
def llm(prompt, stop=["\n"]):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=100,
        # top_p=1,
        # sample=False,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=stop,
        seed=12345
    )
    return completion.choices[0].message.content

In [23]:
llm("What is the capital of France? and write me a poem")

'The capital of France is Paris,'

In [42]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

In [47]:
print(env.print_sections('1920'))

*: Events
**: January
**: February
**: March
**: April
**: May
**: June
**: July
**: August
**: September
**: October
**: November
**: December
**: Date unknown
*: Births
**: January
**: February
**: March
**: April
**: May
**: June
**: July
**: August
**: September
**: October
**: November
**: December
**: Date unknown
*: Deaths
**: January
**: February
**: March
**: April
**: May
**: June
**: July
**: August
**: September
**: October
**: November
**: December
**: Date unknown
*: Nobel Prizes
*: References
**: Primary sources and year books



In [7]:
# additional examples
# read text file
with open("bonn.txt", "r") as file:
    additional_examples = ''.join( file.readlines())
    # examples = [x.strip() for x in examples]

In [8]:
print(additional_examples)

Question: What is the population of the city where the Akademisches Kunstmuseum is?
Thought 1: It seems like you want to know the population of the city where the Akademisches Kunstmuseum is located. Let's start by searching for the Akademisches Kunstmuseum to find out which city it is in.
Action 1: Search[Akademisches Kunstmuseum]
Observation 1: Akademisches Kunstmuseum (Academic Art Museum) is an art museum in Bonn, Germany. It is one of the oldest museums in Bonn and houses the antique collection of the University of Bonn with more than 2,700 plaster casts of antique statues and reliefs, and over 25,000 originals. It is located in a neoclassical building at the southern end of the Hofgarten, near the Electoral Palace. During the renovation of the historic building, the museum can currently be visited at Römerstraße 164 in 53117 Bonn.. The museum was founded in 1818 and has one of the largest collections of plaster casts of ancient Greek and Roman sculptures in the world.
Thought 2: 

## Interactive demo

In [52]:
# test environment manually

question = env.reset(idx=0)
print(question)

for i in range(10):
    thought_str = f'Thought {i+1}:'
    thought = input(thought_str)
    print(thought_str, thought)

    action_str = f"Action {i+1}:"
    action = input(action_str)
    print(action_str, action)
    
    obs, r, done, info = step(env, action[0].lower() + action[1:])
    print(f"Observation {i+1}:", obs)

    if done:
        break
if not done:
    obs, r, done, info = step(env, "finish[]")

Question: Were Scott Derrickson and Ed Wood of the same nationality?
Thought 1: let's first figure out Scott Derrickson's nationality
Action 1: search[scott derrickson]
Observation 1: Scott Derrickson (born July 16, 1966) is an American filmmaker. He is best known for his work in the horror genre, directing films such as The Exorcism of Emily Rose (2005), Sinister (2012) and The Black Phone (2021). He is also known for the superhero film Doctor Strange (2016), based on the Marvel Comics character.. Scott Derrickson grew up in Denver, Colorado. He graduated from Biola University with a B.A.
Thought 2: we need to figure out ed woods nationality
Action 2: search[ed wood]
Observation 2: Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, screenwriter, and pulp novel author.. In the 1950s, Wood directed several low-budget science fiction, crime and horror films that later became cult classics, notably Glen or Glenda (1953), Jail Bait (1954), Bride 

IndexError: string index out of range

In [53]:
info

{'steps': 4, 'answer': None}

# ReAct

In [36]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
(4) PrintSections[entity], which returns the sections of the Wikipedia page for the entity.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples # + additional_examples + '\n'


In [37]:
print(webthink_examples)


Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions:
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[

In [38]:
print(webthink_prompt)

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
(4) PrintSections[entity], which returns the sections of the Wikipedia page for the entity.
Here are some examples.

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (

In [39]:

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [40]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
# for i in idxs[:10]:
i = 6554
r, info = webthink(i, to_print=True)
rs.append(info['em'])
infos.append(info)
print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
print('-----------')
print()

6554 Question: What is the population of the city where the Akademisches Kunstmuseum is?
Thought 1: It seems like you have provided a detailed structure for solving question answering tasks with interleaving Thought, Action, and Observation steps. Let's continue with the next step:

Thought 1: I need to search for the Akademisches Kunstmuseum to find the city where it is located and then determine the population of that city.
Action 1: Search[Akademisches Kunstmuseum]
Observation 1: Akademisches Kunstmuseum (Academic Art Museum) is an art museum in Bonn, Germany. It is one of the oldest museums in Bonn and houses the antique collection of the University of Bonn with more than 2,700 plaster casts of antique statues and reliefs, and over 25,000 originals. It is located in a neoclassical building at the southern end of the Hofgarten, near the Electoral Palace. During the renovation of the historic building, the museum can currently be visited at Römerstraße 164 in 53117 Bonn.. The museum 

In [30]:
print(info['traj'])

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern se

In [31]:
r2, info2 = webthink(i, to_print=True)

6554 Question: What is the population of the city where the Akademisches Kunstmuseum is?
ohh... It seems like you want to know the population of the city where the Akademisches Kunstmuseum is located. Let's start by searching for the Akademisches Kunstmuseum to find out which city it is in.
Thought 1: It seems like you want to know the population of the city where the Akademisches Kunstmuseum is located. Let's start by searching for the Akademisches Kunstmuseum to find out which city it is in.
Action 1: Search[Akademisches Kunstmuseum]
Observation 1: Akademisches Kunstmuseum (Academic Art Museum) is an art museum in Bonn, Germany. It is one of the oldest museums in Bonn and houses the antique collection of the University of Bonn with more than 2,700 plaster casts of antique statues and reliefs, and over 25,000 originals. It is located in a neoclassical building at the southern end of the Hofgarten, near the Electoral Palace. During the renovation of the historic building, the museum ca

In [34]:
print(info2['traj'])

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern se

In [35]:
# print differences in the trajectories
import difflib
d = difflib.Differ()
diff = d.compare(info['traj'].splitlines(), info2['traj'].splitlines())
print('\n'.join(diff))


  Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
  (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
  (2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
  (3) Finish[answer], which returns the answer and finishes the task.
  Here are some examples.
  
  Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
  Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
  Action 1: Search[Colorado orogeny]
  Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
  Thought 2: It does not 

In [25]:
# dump the results using pretty json
import json
with open('results/hotpotqa-gpt-3.5-turbo-examples10.json', 'w') as f:
    json.dump(infos, f, indent=4)

## New action examples

In [ ]:
# 